In [3]:
from astropy.table import QTable
import astropy.units as u
from copy import deepcopy
import numpy as np
import mceh.find_luminosity as fl # v2.3
import astropy.cosmology.units as cu
from mceh import fitting # v2.3

Call manual() to get the manual.


In [4]:
ctp = QTable.read('data/eFEDS_c001_main_ctp_clus_v2.1.fits')
rd = QTable.read('data/modified_random_ver1.fits')
mt = QTable.read('data/eFEDSmasstable.fits')
hsc = QTable.read('data/modified_hsc_ver1.fits')

In [5]:
len(rd)

45483143

In [6]:
new_ctp = QTable()

In [7]:
ctp.colnames

['Name',
 'ID_SRC',
 'RA_CORR',
 'DEC_CORR',
 'RADEC_ERR_CORR',
 'EXT',
 'EXT_ERR',
 'EXT_LIKE',
 'ML_CTS',
 'ML_CTS_ERR',
 'ML_RATE',
 'ML_RATE_ERR',
 'ML_FLUX',
 'ML_FLUX_ERR',
 'DET_LIKE',
 'ML_BKG',
 'ML_EXP',
 'inArea90',
 'F_CONT_BEST_COMB',
 'Z_BEST_COMB',
 'SIGMA_Z_BEST_COMB',
 'LAMBDA_BEST_COMB',
 'ELAMBDA_BEST_COMB',
 'SURV_BEST_COMB',
 'RA_OPTCEN_BEST_COMB',
 'DEC_OPTCEN_BEST_COMB',
 'SPEC_Z_BEST_COMB',
 'N_SPEC',
 'MASSPROX_BEST_COMB',
 'F_CONT_2BEST_COMB',
 'Z_2BEST_COMB',
 'LAMBDA_2BEST_COMB',
 'ELAMBDA_2BEST_COMB',
 'SIGMA_Z_2BEST_COMB',
 'SURV_2BEST_COMB',
 'MASSPROX_2BEST_COMB',
 'FLAG_OPTICAL_X_POS',
 'MASKFRAC_3_FOOT',
 'MASKFRAC_3_FLAGGED',
 'MASKFRAC_R500_FOOT_BEST_COMB',
 'MASKFRAC_R500_FLAGGED_BEST_COMB',
 'MASKFRAC_R500_FOOT_2BEST_COMB',
 'MASKFRAC_R500_FLAGGED_2BEST_COMB',
 'RA_GDMAP_2BEST_COMB',
 'DEC_GDMAP_2BEST_COMB',
 'Alpha',
 'Delta',
 'Beta',
 'Ellip',
 'Centershift',
 'DIST_NEXT_OPT_R500',
 'DISTNEXT_XCLUST_MPC',
 'DISTNEXT_XCLUST_R500',
 'MCMF_NWAY_SEC

In [8]:
# Check units.
ctp['RA_CORR'][0]

144.3035678286463

In [9]:
new_ctp['Name'] = ctp['Name']
new_ctp['ID_SRC'] = ctp['ID_SRC']
new_ctp['RA_CORR'] = ctp['RA_CORR'] * u.deg
new_ctp['DEC_CORR'] = ctp['DEC_CORR'] * u.deg
new_ctp['Z_BEST_COMB'] = ctp['Z_BEST_COMB']

In [10]:
# Add the low contamination flag.
is_l_det = ctp['DET_LIKE'] > 5
is_l_ext = ctp['EXT_LIKE'] > 6
is_ext = ctp['EXT'] > 0
is_f_cont = ctp['F_CONT_BEST_COMB'] < 0.3
z = ctp['Z_BEST_COMB']
is_z = (z > 0.1) & (z < 1.2)
new_ctp['low_cont_flag'] = (is_l_det & is_l_ext & is_ext & is_f_cont & is_z)
len(np.where(new_ctp['low_cont_flag'])[0])

455

In [11]:
# Check the correspondence of masstable
print((mt['Name'] == new_ctp['Name']).all())
mass = mt['median_500c_lcdm']
r = mt['r500c_lcdm']
# Check units.
print(mass[0])
print(r[0], r[0].unit == u.arcmin)

True
13.741195689347354 Msun/h
2.583517430533199 arcmin True


In [12]:
u.add_enabled_units(cu.littleh)

In [13]:
new_ctp['median_500c_lcdm'] = mass.value * (u.M_sun / cu.littleh)
new_ctp['r500c_lcdm'] = r

In [14]:
area = 2 * np.pi * (1 - np.cos(r)) * u.rad**2
new_ctp['area'] = area.to(u.arcmin**2)
all_area = []
for i in range(len(new_ctp)):
    all_area.append(fl.unmasked_area(new_ctp['RA_CORR'][i], new_ctp['DEC_CORR'][i], r[i], rd))
new_ctp['unmasked_area'] = [_.value for _ in all_area] * all_area[0].unit

In [15]:
new_ctp['unmasked_area']

<Quantity [1.71300e+01, 8.56000e+00, 1.95400e+01, 3.71500e+01, 8.21300e+01,
           0.00000e+00, 3.24400e+01, 1.09299e+03, 3.82100e+01, 3.43000e+01,
           1.13700e+01, 3.94300e+01, 1.19200e+01, 2.00120e+02, 4.94400e+01,
           2.49300e+01, 2.35000e+01, 8.52200e+01, 1.93400e+01, 8.40000e-01,
           3.18700e+01, 4.82200e+01, 3.98900e+01, 2.83900e+01, 2.64100e+01,
           1.15000e+00, 1.85400e+01, 3.21000e+01, 2.06700e+01, 3.79900e+01,
           2.24900e+01, 3.49400e+01, 1.23200e+01, 0.00000e+00, 1.27400e+01,
           0.00000e+00, 7.25000e+00, 2.15500e+01, 3.65800e+01, 1.77400e+01,
           2.68600e+01, 1.84400e+01, 2.01400e+01, 1.89600e+01, 7.46000e+00,
           0.00000e+00, 1.28200e+01, 2.94400e+01, 1.65800e+01, 3.18200e+01,
           5.38000e+00, 1.42100e+01, 2.21900e+01, 3.05300e+01, 1.38200e+01,
           4.68500e+01, 1.11880e+02, 0.00000e+00, 1.42700e+01, 0.00000e+00,
           2.04100e+01, 1.68300e+01, 2.34600e+01, 5.70000e+00, 2.82300e+01,
           1

In [16]:
new_ctp['area']

<Quantity [  20.96875495,   25.97175197,   26.56712972,   42.6935195 ,
             98.68232259,   33.58541037,   39.56980444, 1404.56108842,
             73.19150623,   61.09131689,   12.89770251,   44.45901422,
             15.57739598,  264.4898177 ,   56.94599058,   28.88032823,
             28.47453068,  124.65764141,   27.70035882,   57.29507785,
             52.93613456,   60.94377771,   50.90206075,   97.15897663,
             33.05656832,    7.69634851,   31.07086987,   39.09630195,
             22.59475917,   47.69633248,   25.45895423,   43.32519986,
             13.47224835,   21.57965865,   18.21777932,   93.06405094,
             10.31766181,   24.75765603,   43.58435085,   20.70218271,
             40.92388524,   22.4549979 ,   29.63609912,   22.93494775,
              9.05540476,   37.73818235,   19.5907534 ,   34.18960755,
             25.34003757,   37.67496258,   14.75198858,   18.64816698,
             30.28785153,   34.01609113,   18.35441454,   59.32358378,
      

In [17]:
new_ctp['i_cmag'] = fitting.cmag(new_ctp['Z_BEST_COMB'])

In [18]:
from rsmodel import outputModels
@np.vectorize
def cmag(z, band_name):  # i-band characteristic magnitude by the model
    # ex: hsc_i
    return outputModels.printRSmodel('data/bc03_rs_zfp3d0_tau_p0d4.fits',
                                     [band_name], z)[1][band_name][4]

In [19]:
z = new_ctp['Z_BEST_COMB']
new_ctp['g_cmag'] = cmag(z, 'hsc_g')
new_ctp['r_cmag'] = cmag(z, 'hsc_r')
new_ctp['z_cmag'] = cmag(z, 'hsc_z')
new_ctp['y_cmag'] = cmag(z, 'hsc_y')

In [45]:
#i_obs_alllf_raw2
galaxy_index = []
for i in range(len(new_ctp)):
    galaxy_index.append(fl.index_within_angular_radius(hsc['ra'], hsc['dec'], 
                                   new_ctp['RA_CORR'][i], 
                                   new_ctp['DEC_CORR'][i], 
                                   new_ctp['r500c_lcdm'][i]))
    print(f'Finish {i+1}/{len(new_ctp)}')

Finish 1/542
Finish 2/542
Finish 3/542
Finish 4/542
Finish 5/542
Finish 6/542
Finish 7/542
Finish 8/542
Finish 9/542
Finish 10/542
Finish 11/542
Finish 12/542
Finish 13/542
Finish 14/542
Finish 15/542
Finish 16/542
Finish 17/542
Finish 18/542
Finish 19/542
Finish 20/542
Finish 21/542
Finish 22/542
Finish 23/542
Finish 24/542
Finish 25/542
Finish 26/542
Finish 27/542
Finish 28/542
Finish 29/542
Finish 30/542
Finish 31/542
Finish 32/542
Finish 33/542
Finish 34/542
Finish 35/542
Finish 36/542
Finish 37/542
Finish 38/542
Finish 39/542
Finish 40/542
Finish 41/542
Finish 42/542
Finish 43/542
Finish 44/542
Finish 45/542
Finish 46/542
Finish 47/542
Finish 48/542
Finish 49/542
Finish 50/542
Finish 51/542
Finish 52/542
Finish 53/542
Finish 54/542
Finish 55/542
Finish 56/542
Finish 57/542
Finish 58/542
Finish 59/542
Finish 60/542
Finish 61/542
Finish 62/542
Finish 63/542
Finish 64/542
Finish 65/542
Finish 66/542
Finish 67/542
Finish 68/542
Finish 69/542
Finish 70/542
Finish 71/542
Finish 72/542
F

In [46]:
new_ctp['galaxy_index'] = galaxy_index

In [47]:
new_ctp.write('data/modified_efeds_ver6.fits')